In [1]:
import numpy as np
import cv2

In [2]:
# 读取视频
cap = cv2.VideoCapture('./test.avi')

# 读第一帧图片
ret, old_frame = cap.read()
# 变成黑白
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
# 检测角点
feature_params = dict(
    maxCorners=100,
    qualityLevel=0.3,
    minDistance=7
)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

# 创建一个mask
mask = np.zeros_like(old_frame)
# 随机颜色
color = np.random.randint(0, 255, (100, 3))

while True:
    ret, frame = cap.read()
    if frame is None:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # 光流估计
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, gray, p0, None, winSize=(15, 15), maxLevel=2)
    
    # 哪些特征点找到了, 哪些特征点没找到.
    good_new = p1[st==1]
    good_old = p0[st==1]
    
    # 绘制轨迹
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (a, b), (c, d), color[i].tolist(), 2)
        frame = cv2.circle(frame, (a, b), 5, color[i].tolist(), -1)
        
    img = cv2.add(frame, mask)
    
    cv2.imshow('frame', img)
    key = cv2.waitKey(150)
    if key == 27:
        break
        
    # 更新
    old_gray = gray.copy()
    p0 = good_new.reshape(-1, 1, 2)
    
cv2.destroyAllWindows()
cap.release()

In [ ]:
FlowNet, 2015